<a href="https://colab.research.google.com/github/jiin124/Introduction-to-NLP-using-deep-learning/blob/main/PyTorch/CNN%EC%9C%BC%EB%A1%9C_MNIST_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 모델 이해하기

합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 조금 다름

## 1. 첫번째 표기방법

합성곱(nn.Con2d)+활성화함수(nn.ReLU) 를 하나의 합성곱 층으로 보고 맥스풀링은 풀링층으로 별도 명명

## 2. 두번째 표기방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 봅니다.



# 2. 모델 구현하기

## 필요한 도구들을 임포트

In [1]:
import torch
import torch.nn as nn

In [2]:
#임의의 텐서 만든리 1*1*28*28
inputs=torch.Tensor(1,1,28,28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


## 합성곱층과 풀링 선언

In [4]:
#1채널로 32채널을 뽑는데 커널 사이즈는 3, 패딩은 1
conv1=nn.Conv2d(1,32,3,padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
#두번째 합성곱층 구현
#32채널을 입력받아 64채널을 뽑는데, 커널 사이즈는 3 패딩은 1
conv2=nn.Conv2d(32,64,kernel_size=3,padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
#맥스풀링 구현
#커널=스트라이드
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 구현체를 연결해 모델 만들기

In [7]:
#첫번째 합성곱층 통과
out=conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [8]:
#맥스풀링 통과
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [9]:
#두번째 합성곱층 통과
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


14*14 로 변함이 없는 이유는 패딩을 1폭으로 하고 3*3커널을 사용하면 크기가 보존되기 때문. 

In [10]:
#맥스풀링을 통과
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [11]:
#out의 첫번째 차원
out.size(0)

1

In [12]:
#두번째 차원
out.size(1)

64

In [13]:
#세번째 차원
out.size(2)

7

In [14]:
#네번째 차원
out.size(3)

7

In [15]:
#view를 사용해 텐서를 펼치는 작업
out=out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


 전결합층(Fully-Connteced layer)를 통과. 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환.

In [16]:
fc=nn.Linear(3136,10)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


# 3. CNN으로 MNIST 분류

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [18]:
device='cuda' if torch.cuda.is_available() else 'cpu'

#랜덤시드고정
torch.manual_seed(777)

#gpu 사용가능일 경우 랜덤 시드 고정
if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [19]:
#학습에 사용할 파라미터 설정
learning_rate=0.001
training_epochs=15
batch_size=100

In [20]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)

mnist_test=dsets.MNIST(root='MINST_data/',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting MINST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MINST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MINST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MINST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MINST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MINST_data/MNIST/raw



In [21]:
#데이터로더로 배치크기 지정~
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [24]:
#이제 클래스로 모델 설계~
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    #첫번째층
    # ImgIn shape=(?, 28, 28, 1)
    #    Conv     -> (?, 28, 28, 32)
    #    Pool     -> (?, 14, 14, 32)
    self.layer1=torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    #두번째층
    # ImgIn shape=(?, 14, 14, 32)
    #    Conv      ->(?, 14, 14, 64)
    #    Pool      ->(?, 7, 7, 64)
    self.layer2=torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )
    # 전결합층 7x7x64 inputs -> 10 outputs
    self.fc=torch.nn.Linear(7*7*64,10,bias=True)

    # 전결합층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0),-1)
    out=self.fc(out)
    return out




In [25]:
#CNN모델 정의
model=CNN().to(device)

In [26]:
#비용함수 옵티마이저
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [27]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [29]:
for epoch in range(training_epochs):
  avg_cost=0

  for X,Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    hypothesis=model(X)
    cost=criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.214077562
[Epoch:    2] cost = 0.0625753999
[Epoch:    3] cost = 0.0468574911
[Epoch:    4] cost = 0.0353891365
[Epoch:    5] cost = 0.0304739773
[Epoch:    6] cost = 0.0253360011
[Epoch:    7] cost = 0.0222034641
[Epoch:    8] cost = 0.0184198748
[Epoch:    9] cost = 0.0143118249
[Epoch:   10] cost = 0.012750579
[Epoch:   11] cost = 0.0114071816
[Epoch:   12] cost = 0.00942197349
[Epoch:   13] cost = 0.00787676126
[Epoch:   14] cost = 0.00546959881
[Epoch:   15] cost = 0.00633426709


In [30]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9857000112533569
